In [1]:
import splinepy
import numpy as np
import vedo
from sdf_microstructure import (
    create_microstructure_from_experiment,
    export_mesh,
    tetrahedralize_surface,
)

from deep_sdf.analysis import linear_elasticity as le

vedo.settings.default_backend = "k3d"

test_mesh = False

In [2]:
faces, jacobian = create_microstructure_from_experiment("/home/michael.kofler/DeepSDF/experiments/round_cross_big_network", tiling=[2,1,1], N_base=20)

In [3]:
if test_mesh:
    rect = splinepy.helpme.create.box(2, 1, 1)
    volumes = rect.extract.volumes(resolution=30)
else:
    volumes = tetrahedralize_surface(faces)

Delaunizing vertices...
Delaunay seconds:  0.034453
Creating surface mesh ...
Surface mesh seconds:  0.010999
Recovering boundaries...
Boundary recovery seconds:  0.033622
Removing exterior tetrahedra ...
Exterior tets removal seconds:  0.008187
Recovering Delaunayness...
Delaunay recovery seconds:  0.023233
Improving mesh...
Mesh improvement seconds:  0.006575

Writing nodes.
Writing elements.
Writing faces.
Writing edges.

Output seconds:  0.001282
Total running seconds:  0.118675

Statistics:

  Input points: 6794
  Input facets: 13608
  Input segments: 9432
  Input holes: 0
  Input regions: 0

  Mesh points: 6794
  Mesh tetrahedra: 21411
  Mesh faces: 49626
  Mesh faces on exterior boundary: 13608
  Mesh faces on input facets: 13608
  Mesh edges on input segments: 9432



In [4]:
export_mesh(volumes, "test_mesh.mesh", show_mesh=False)
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")
le_problem.show_mesh()

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [5]:
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up()
dVertices = np.float64(jacobian[:,:,0]*0.1)
vol, der_vol = le_problem.compute_volume(dTheta=dVertices)
print(f"Vol: {vol:.5g}, dVol: {der_vol:.5g}")
le_problem.solve()
compliance, der_compliance = le_problem.compute_compliance(dTheta=dVertices)
print(f"Compliance: {compliance:.5g}, dCompliance: {der_compliance:.5g}")
le_problem.show_solution(output=["u_vec","strain_energy_density"], vmax=0.2)

Vol: 0.81736, dVol: -0.020616
Finished Solution. Max deflection: 0.2132845927664023
Saving results to linear_elasticity
Compliance: 2653.6, dCompliance: 0.012185


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [6]:
le_problem = le.LinearElasticityProblem()
volumes_stretched = volumes.copy()
volumes_stretched.vertices = volumes.vertices + dVertices
export_mesh(volumes_stretched, "test_mesh_stretched.mesh")
le_problem.read_mesh("test_mesh_stretched.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up(ref_levels=0)
vol, _ = le_problem.compute_volume()
print(f"Volume of deformed mesh {vol:.5g}")
le_problem.solve()
compliance, der_compliance = le_problem.compute_compliance()
print(f"Compliance of deformed mesh: {compliance:.5g}")
le_problem.show_solution(output=["u_vec","strain_energy_density"])

Elements with wrong orientation: 301 / 21411 (fixed)
Volume of deformed mesh 0.80005
Finished Solution. Max deflection: 0.2193602606143406
Saving results to linear_elasticity
Compliance of deformed mesh: 2800.5


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…